In [1]:
import datetime
import requests
import io
import pandas as pd
import holidays

Get Data

In [2]:
##### SET START DATE | http://web.mta.info/developers/turnstile.html
date_val = datetime.date(2018, 9, 1)

dfs = []

##### SET RANGE for number of weeks of data needed
for x in range(1):
    
    url = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt'
    date_val_str = date_val.strftime("%y%m%d") # convert datetime format to string
    
    # create url from pieces above
    full_url = url.format(date_val_str)
    
    # Get data from url
    r = requests.get(full_url)
    r.raise_for_status()
    
    # Create dataframe
    r_content = r.content
    df_mta_data = pd.read_csv(io.StringIO(r_content.decode('utf-8')))
    dfs.append(df_mta_data)
    
    # Keep header if it's the first one, otherwise remove header and append to csv directly
    #if x == 0:
    #    df_mta_data.to_csv(output_file_loc, index = False, header = True)
            
    #else:
    #    with open(output_file_loc, 'a') as f:
    #        df_mta_data.to_csv(output_file_loc, mode = 'a', header = False, index = False)

    # Add 7 days to the date         
    date_val = date_val + datetime.timedelta(days = 7)
    
    # Just to make sure it's running smoothly
    print("Done with: " + full_url)
    print(x)
    
df = pd.concat(dfs)
print("yass done done done!!")    

Done with: http://web.mta.info/developers/data/nyct/turnstile/turnstile_180901.txt
0
yass done done done!!


Clean Data

In [3]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/25/2018,00:00:00,REGULAR,6736067,2283184
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/25/2018,04:00:00,REGULAR,6736087,2283188
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/25/2018,08:00:00,REGULAR,6736105,2283229
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/25/2018,12:00:00,REGULAR,6736180,2283314
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/25/2018,16:00:00,REGULAR,6736349,2283384


In [4]:
#clean up the columns
df.columns = [col.strip() for col in df.columns]

In [5]:
#Clean up the columns
df.columns = [col.strip() for col in df.columns]

#Add datetime column
df['DATE_TIME'] = df['DATE'] + " " + df['TIME']
df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'])

#Add Day of Week
df['DAY_OF_WEEK'] = (df['DATE_TIME']
                     .dt.dayofweek.map({0: "Monday", 
                                        1: "Tuesday", 
                                        2: "Wednesday", 
                                        3: "Thursday", 
                                        4: "Friday", 
                                        5: "Saturday", 
                                        6:"Sunday"}))

#Add Month
df['MONTH'] = (pd.DatetimeIndex(df['DATE_TIME'])
               .month.map({1:'January', 
                        2: 'February',
                        3: 'March',
                        4: 'April',
                        5: 'May',
                        6: 'June',
                        7: 'July', 
                        8: 'August', 
                        9: 'Septemner', 
                        10: 'October', 
                        11: 'November', 
                        12: 'December'}))

#Add Year
df['YEAR'] = pd.DatetimeIndex(df['DATE_TIME']).year

#Add Holiday
us_holidays = holidays.UnitedStates(years = [2018, 2019])
df['HOLIDAY'] = pd.to_datetime(df['DATE']).isin(us_holidays)

In [6]:
len(df)

197625

In [7]:
#Get rid of duplicates
df = df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])

In [9]:
#Sanity check that dupes are removed
(df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(5)

,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES
0,A002,R051,02-00-00,59 ST,2018-08-25 00:00:00,1
131814,R147,R033,04-00-06,TIMES SQ-42 ST,2018-08-27 04:00:00,1
131744,R147,R033,04-00-04,TIMES SQ-42 ST,2018-08-29 08:00:00,1
131745,R147,R033,04-00-04,TIMES SQ-42 ST,2018-08-29 12:00:00,1
131746,R147,R033,04-00-04,TIMES SQ-42 ST,2018-08-29 16:00:00,1


Filter Data

In [14]:
def select_time(df, start_time_hour=6, end_time_hour=12):
    df['HOUR'] = pd.to_datetime(df['TIME']).dt.hour
    df_time_frame = df[df["HOUR"].isin(list(range(start_time_hour, end_time_hour + 1)))]
    
    assert len(
        df_time_frame["HOUR"]
        .value_counts()
        .index
        .isin(list(range(start_time_hour, end_time_hour + 1)))
    ) == (end_time_hour + 1 ) - (start_time_hour)
    
    return df_time_frame

In [44]:
df_morn = select_time(df)

In [45]:
# sort values
df_morn.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], 
                          inplace=True, ascending=True)
# df_morn.head(40)
df_morn['for_merging'] = df['C/A'] + "_" + df['UNIT'] + "_" + df['SCP'] + "_" + df['STATION'] + "_" + df['DATE']

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [65]:
df_first = (df_morn
            .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"],as_index=False)
            .head(1))

df_last = (df_morn
           .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"],as_index=False)
           .tail(1)[['ENTRIES', 'EXITS', 'for_merging']]).rename(columns={'ENTRIES': 'LAST_ENTRIES', 'EXITS': 'LAST_EXITS'})


In [ ]:
df_first

In [64]:
del(df_last)
del(df_first)

In [66]:
df_daily = pd.merge(df_first, df_last, on = 'for_merging', how = 'left')

In [71]:
cols = ['C/A', 
        'UNIT', 
        'SCP', 
        'STATION', 
        'DATE', 
        'DAY_OF_WEEK', 
        'MONTH', 
        'YEAR', 
        'HOLIDAY', 
        'ENTRIES', 
        'EXITS', 
        'LAST_ENTRIES', 
        'LAST_EXITS']

In [73]:
def get_daily_entry_counts(row, max_counter):
    counter = row["ENTRIES"] - row["LAST_ENTRIES"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        print(row["ENTRIES"], row["LAST_ENTRIES"])
        counter = min(row["ENTRIES"], row["LAST_ENTRIES"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

In [ ]:
def get_daily_exit_counts(row, max_counter):
    counter = row["EXITS"] - row["LAST_EXITS"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        print(row["EXITS"], row["LAST_EXITS"])
        counter = min(row["EXITS"], row["LAST_EXITS"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

In [74]:
df_daily["DAILY_ENTRIES"] = df_daily.apply(get_daily_entry_counts, axis=1, max_counter=1000000)

1097772 5091


In [79]:
df_daily["DAILY_ENTRIES"].mean()

163.16182483322575

In [80]:
df_daily["DAILY_ENTRIES"].max()

419042

In [82]:
df_daily[df_daily["DAILY_ENTRIES"] == 419042]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,...,DATE_TIME,DAY_OF_WEEK,MONTH,YEAR,HOLIDAY,hour,for_merging,LAST_ENTRIES,LAST_EXITS,DAILY_ENTRIES
18520,PTH05,R543,00-00-05,EXCHANGE PLACE,1,PTH,08/30/2018,07:14:10,REGULAR,419044,...,2018-08-30 07:14:10,Thursday,August,2018,False,7,PTH05_R543_00-00-05_EXCHANGE PLACE_08/30/2018,2,319,419042


In [72]:
df_daily[cols]

,C/A,UNIT,SCP,STATION,DATE,DAY_OF_WEEK,MONTH,YEAR,HOLIDAY,ENTRIES,EXITS,LAST_ENTRIES,LAST_EXITS
0,A002,R051,02-00-00,59 ST,08/25/2018,Saturday,August,2018,False,6736105,2283229,6736180,2283314
1,A002,R051,02-00-00,59 ST,08/26/2018,Sunday,August,2018,False,6736705,2283483,6736746,2283524
2,A002,R051,02-00-00,59 ST,08/27/2018,Monday,August,2018,False,6737169,2283736,6737301,2283965
3,A002,R051,02-00-00,59 ST,08/28/2018,Tuesday,August,2018,False,6738474,2284216,6738621,2284402
4,A002,R051,02-00-00,59 ST,08/29/2018,Wednesday,August,2018,False,6739847,2284600,6739986,2284838
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32524,TRAM2,R469,00-05-01,RIT-ROOSEVELT,08/27/2018,Monday,August,2018,False,5554,348,5554,348
32525,TRAM2,R469,00-05-01,RIT-ROOSEVELT,08/28/2018,Tuesday,August,2018,False,5554,348,5554,348
32526,TRAM2,R469,00-05-01,RIT-ROOSEVELT,08/29/2018,Wednesday,August,2018,False,5554,348,5554,348
32527,TRAM2,R469,00-05-01,RIT-ROOSEVELT,08/30/2018,Thursday,August,2018,False,5554,348,5554,348


In [ ]:
pd.merge(df_first, df_last, on = )

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 